This notebook demonstrates how to apply an effect to a map. In this example, we are creating a grid layer with 3D scaling, then applying a light and shadow effect. 

## Dependencies

This notebook requires the following Python dependencies:

- `unfolded.map-sdk`: The Studio Map SDK
- `pandas`: DataFrame library
- `requests`: HTTP request library for retrieving data from a URL

The next cell will install these dependencies.

In [ ]:
# If in Colab, install this notebook's required dependencies
import sys
if "google.colab" in sys.modules:
    !pip install 'unfolded.map_sdk>=1.0' requests pandas

## Imports

In [17]:
from unfolded.map_sdk import create_map
import pandas as pd
import requests

## Retrieving Data

This dataset contains earthquake data for the pacific region. Timestamps, lat/lng locations, as well as magnitude and depth are provided. The data is then loaded into a dataframe `df`.

In [18]:
# Define the URL of the JSON file
url = "https://4sq-studio-public.s3.us-west-2.amazonaws.com/sdk/examples/sample-data/earthquakes.json"

# Make a GET request to fetch the data
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Load the JSON data into a DataFrame
    data = response.json()
    df = pd.DataFrame(data)
    print("Data successfully loaded into DataFrame.")
else:
    print("Failed to fetch JSON data. Status code:", response.status_code)


Data successfully loaded into DataFrame.


## Create Map

Before we can do anything, let's create and render the map. In most ipython eenvironments, this map will be updated whenever changes are made. However, environments such as databricks require the map to be re-rendered to view any changes.

In [19]:
map = create_map()
map

SyncWidgetMap(style={'height': '100%', 'width': '100%'})

## Add Data

With the map created, we can now add the dataframe to the map as a dataset:

In [20]:
map.add_dataset({    
    "id": "earthquakes",
    "label": "Earthquakes",
    "data": df
},
    auto_create_layers = False
)

LocalDataset(id='earthquakes', type=<DatasetType.LOCAL: 'local'>, label='Earthquakes', color=(0, 92, 255), fields=[BasicField(name='column_0', label='column_0', type=<BasicFieldType.INTEGER: 'integer'>), TimestampField(name='DateTime', label='DateTime', type=<TimestampFieldType.TIMESTAMP: 'timestamp'>, time_format='YYYY/M/D HH:mm:ss.SSSS'), BasicField(name='Latitude', label='Latitude', type=<BasicFieldType.REAL: 'real'>), BasicField(name='Longitude', label='Longitude', type=<BasicFieldType.REAL: 'real'>), BasicField(name='Depth', label='Depth', type=<BasicFieldType.REAL: 'real'>), BasicField(name='Magnitude', label='Magnitude', type=<BasicFieldType.REAL: 'real'>), BasicField(name='MagType', label='MagType', type=<BasicFieldType.STRING: 'string'>), BasicField(name='NbStations', label='NbStations', type=<BasicFieldType.INTEGER: 'integer'>), BasicField(name='Gap', label='Gap', type=<BasicFieldType.INTEGER: 'integer'>), BasicField(name='Distance', label='Distance', type=<BasicFieldType.INT

## Create Layer

With the dataset added, we can create a new layer. This layer uses a grid, that, by default aggregates the number of points contained within each square's region. We enable 3D and provide an elevation scale to accentuate height.

In [21]:
map.add_layer({
    "type":"grid",
    "data_id":"earthquakes",
    "fields": {
      "lat": "Latitude",
      "lng": "Longitude"
    },
    "config": {
        "visConfig": {
          "worldUnitSize": 26,
          "elevationScale": 90,
          "enable3d": True
        }
    }
})

Layer(type=<LayerType.GRID: 'grid'>, data_id='earthquakes', fields={'lat': 'Latitude', 'lng': 'Longitude'}, label='new layer', is_visible=True, config=LayerConfig(visual_channels={'colorField': None, 'colorScale': 'quantile', 'sizeField': None, 'sizeScale': 'linear'}, vis_config={'opacity': 0.8, 'worldUnitSize': 26, 'colorRange': {'name': 'Global Warming', 'type': 'sequential', 'category': 'Uber', 'colors': ['#4C0035', '#880030', '#B72F15', '#D6610A', '#EF9100', '#FFC300']}, 'coverage': 1, 'sizeRange': [0, 500], 'percentile': [0, 100], 'elevationPercentile': [0, 100], 'elevationScale': 90, 'enableElevationZoomFactor': True, 'fixedHeight': False, 'colorAggregation': 'count', 'sizeAggregation': 'count', 'enable3d': True}, column_mode=None), id='0dd7f4aa-5725-4448-b113-fe09ed6cc5b7')

## Set Map Configuration

The following calls set the map's state and view for optimal 3D viewing:

In [22]:
map.set_map_config({
  "version": "v1",
  "config": {
    "mapState": {
        "pitch": 50,
        "bearing": 24,
        "mapViewMode": "MODE_3D",
        
    }
  }
})

map.set_view({
  "latitude": 36.7045671093519,
  "longitude": -122.47582941779496,
  "zoom": 5.920306814575524
})

View(longitude=-122.47582941779496, latitude=36.7045671093519, zoom=5.920306814575524, pitch=50, bearing=24)

Let's render the map before adding any effects:

In [24]:
map

SyncWidgetMap(style={'height': '100%', 'width': '100%'})

## Create Effect

Let's add the light and shadow effect to the map. We will keep the default settings, which attempts to replicate realistic shadowing. To make the shadows clear, we will choose a time where the sun is setting in the West Coast. 

In [25]:
map.add_effect({
  "type": "light-and-shadow",
  "parameters": {
    "timeMode": "pick",
    "timestamp": 1695250260000
  },
})

Effect(is_enabled=True, parameters={'timestamp': 1695250260000, 'timeMode': 'pick', 'shadowIntensity': 0.5, 'shadowColor': [0, 0, 0], 'sunLightColor': [255, 255, 255], 'sunLightIntensity': 1, 'ambientLightColor': [255, 255, 255], 'ambientLightIntensity': 1, 'timezone': 'America/Los_Angeles'}, id='42e13c5e-9653-4230-9e4a-05d4cbf58b78', type=<EffectType.LIGHT_AND_SHADOW: 'light-and-shadow'>)

## Add More Effects

We can stack effects atop one another. For example, let's add a hexagonal pixelatation effect to the map:

In [9]:
map

SyncWidgetMap(style={'height': '100%', 'width': '100%'})

In [31]:
map.add_effect({
  "type": "hexagonal-pixelate",
    "parameters": {
    "scale": 8.5
  },
})
map

SyncWidgetMap(style={'height': '100%', 'width': '100%'})

Let's add a vignette effect as well:

In [34]:
map.add_effect({
  "type": "vignette"
})
map

SyncWidgetMap(style={'height': '100%', 'width': '100%'})

Let's add a final effect, this one altering the hue and saturation on the map. This can be useful for quickly changing the color of the map without jumping into layer settings:

In [35]:
map.add_effect({
  "type": "hue-saturation",
  "parameters": {
    "hue": 0.6,
    "saturation": 0.4
  }
})
map

SyncWidgetMap(style={'height': '100%', 'width': '100%'})